# Spotify Web API Integration with Tekore

## Import Tekore and Spotify authentication

In this notebook we will use Tekore to interact with the Spotify Web API. We want to combine our Last.FM recommender system with Spotify so that we can automatically generate a list of recommended artists based on a user's spotify top artists

In [ ]:
import tekore as tk

Here we enter user details to log in to spotify, we have used #s to anonymise them afterwards

In [ ]:
# Read in keys
client_id = "952362c4671f440ab5024e############"
client_secret = "04cbcba018864e9ba4############"

# Set URI
redirect_uri = 'http://mysite.com/callback/'

In [ ]:
# authenticate
conf = (client_id, client_secret, redirect_uri)
token = tk.prompt_for_user_token(*conf, scope=tk.scope.every)

spotify = tk.Spotify(token)

## Get user's most listened to Spotify Artists

The spotify.current_user_top_artists function will return the top artists that the Spotify user listens to

In [ ]:
artists = spotify.current_user_top_artists(limit=5)
spotify_artists = artists.items

Now that we have the top 5, we can quickly take a look at what our Regularized model recommends. We can see Colter Wall isn't in the dataset, nor are System of a Down. So we will only get results for the Arctic Monkeys, Oasis, The Cranberries.

In [ ]:
for i in range(0,5):
  try: 
    artist_neighbors(reg_model, spotify_artists[i], DOT)
    artist_neighbors(reg_model, spotify_artists[i], COSINE)
  except:
    print(spotify_artists[i] + " isn't in the data")


Colter Wall isn't in the data
Nearest neighbors of : Arctic Monkeys.
[Found more than one matching artist. Other candidates: Arctic Monkeys vs The Killers]


,dot score,names
201,27.652,Arctic Monkeys
59,15.966,Coldplay
148,15.565,Radiohead
418,15.507,The Strokes
197,13.730,Blur
406,13.467,blink-182


Nearest neighbors of : Arctic Monkeys.
[Found more than one matching artist. Other candidates: Arctic Monkeys vs The Killers]


,cosine score,names
201,1.000,Arctic Monkeys
15292,0.773,The Demanders
3426,0.752,"Christina,Lil' Kim,Mya,Pink"
2113,0.745,The Rascals
3420,0.744,London Festival Orchestra/Peter Knight/The Moo...
15293,0.738,Marie & Nick


Nearest neighbors of : Oasis.


...

,cosine score,names
527,1.000,Oasis
1232,0.758,Skank
960,0.725,Kasabian
2519,0.724,Beady Eye
6322,0.689,1990s
7250,0.667,Fountains of Wayne


Nearest neighbors of : The Cranberries.


,dot score,names
160,15.286,The Cranberries
153,11.060,The Cure
608,10.638,Pearl Jam
167,10.082,Placebo
223,10.003,The Killers
709,9.987,R.E.M.


Nearest neighbors of : The Cranberries.


,cosine score,names
160,1.000,The Cranberries
709,0.717,R.E.M.
1374,0.710,Imogen Heap
10878,0.675,Ben Moody
3265,0.672,Phil Collins
10316,0.667,Juli


It's clear that, as we saw in the previous notebook, the recommendations are really quite good for these artists, matching the kinds of artists you'd expect to see.

Now we change the artist_neighbors function so it returns the actual dataframe of recommended artists and doesn't just display them

In [ ]:
def artist_neighbors_return_df(model, title_substring, measure=DOT, k=6):
  # Search for movie ids that match the given substring.
  ids =  artists[artists['name'].str.contains(title_substring)].index.values
  titles = artists.iloc[ids]['name'].values
  if len(titles) == 0:
    raise ValueError("Found no artist with title %s" % title_substring)
  print("Nearest neighbors of : %s." % titles[0])
  if len(titles) > 1:
    print("[Found more than one matching artist. Other candidates: {}]".format(
        ", ".join(titles[1:])))
  artistID = ids[0]
  scores = compute_scores(
      model.embeddings["artistID"][artistID], model.embeddings["artistID"],
      measure)
  score_key = measure + ' score'
  df = pd.DataFrame({
      score_key: list(scores),
      'names': artists['name'],
  })
  return df.sort_values([score_key], ascending=False).head(k)

We store all of the dot product score recommendations and all of the cosine similarity score recommendations

In [ ]:
dot_recs = []
cos_recs = []
all_recs = []

for i in range(0,5):
  try: 
    dot_recs.append(artist_neighbors_return_df(reg_model, spotify_artists[i], DOT))
    cos_recs.append(artist_neighbors_return_df(reg_model, spotify_artists[i], COSINE))
  except:
    print(spotify_artists[i] + " isn't in the data")

Colter Wall isn't in the data
Nearest neighbors of : Arctic Monkeys.
[Found more than one matching artist. Other candidates: Arctic Monkeys vs The Killers]
Nearest neighbors of : Arctic Monkeys.
[Found more than one matching artist. Other candidates: Arctic Monkeys vs The Killers]
Nearest neighbors of : Oasis.
Nearest neighbors of : Oasis.
Nearest neighbors of : The Cranberries.
Nearest neighbors of : The Cranberries.
System Of A Down isn't in the data


In [ ]:
dot_df = pd.concat(dot_recs)
cos_df = pd.concat(cos_recs)

Now we sort these two dataframes by score, descending. So we have the most recommended at the top

In [ ]:
dot_df = dot_df.sort_values(by=['dot score'], ascending=False)
cos_df = cos_df.sort_values(by=['cosine score'], ascending=False)

We add both sets of scores to the same database, and we remove any duplicates and also the user's top artists themselves as these will always be top of these recommendations but this isn't useful. We only want to recommend users that aren't already in the user's top listened to.

In [ ]:
all_recs = []
for i in range(1,len(dot_df)):
  if dot_df.names.iloc[i] not in spotify_artists:
    all_recs.append(dot_df.names.iloc[i])

for i in range(1,len(cos_df)):
  if cos_df.names.iloc[i] not in spotify_artists:
    all_recs.append(cos_df.names.iloc[i])


Add a column for rank, from 1 to the length of the dataframe

In [ ]:
recs_df = pd.DataFrame(all_recs)
recs_df['Rank'] = int
for i in range(0,len(recs_df)):
  recs_df['Rank'][i] = i+1

Rename column to "Recommended Artist"

In [ ]:
recs_df = recs_df.rename(columns={0: "Recommended Artist"})

## Final Result

Now we just make the output look nice, and there we have it. A list of, 30 in this case, of recommended artists based of the user's Spotify account. In this case, the Spotify account belonged to one of our team and they confirmed that this list is really great, showing us that it has worked out pretty well!

In [ ]:
from tabulate import tabulate

print(tabulate(recs_df, headers='keys', tablefmt='psql', showindex="never"))

+--------------------------------------------------------+--------+
| Recommended Artist                                     |   Rank |
|--------------------------------------------------------+--------|
| Pink Floyd                                             |      1 |
| Coldplay                                               |      2 |
| Radiohead                                              |      3 |
| The Strokes                                            |      4 |
| Kasabian                                               |      5 |
| The Beatles                                            |      6 |
| Blur                                                   |      7 |
| blink-182                                              |      8 |
| Muse                                                   |      9 |
| Franz Ferdinand                                        |     10 |
| The Cure                                               |     11 |
| Pearl Jam                                     